# GLoVe

In this task we will implement the GLoVe algorithm for generating word embeddings

We will Game of Thrones dialogue from all seasons as our corpus. Each line consists of a dialogue spoken by a character in a scene.

1) Given the corpus, define a function that removes all the punctuations and stop words from the text. (3 points)

In [ ]:
def normalize_text(corpus):
    # write your code snippet here
    # You can modify the function signature if you want to
    pass

2) From normalized sentences obtained in the previous step, create word-word frequency matrix with all the unique words. You will also need to create a word2index mapping (3 points)

In [ ]:
word2index = {} # you can replace this with a more efficient data structure

def generate_frequency_matrix(corpus, window_size=3):
    # write your code snippet here
    
    pass

3) Define weighting function used in GLoVe. (4 points)

$f(x) = (\frac{x}{x_{max}})^\alpha$ if $x < x_{max}$, 1 otherwise

In [ ]:
def weighting_func(x, x_max=100):
    # write your code snippet here
    
    pass

4) Create the Glove model class using pytorch. (10 points)
   
   Hints: 
   1. The forward pass will compute $W_i^T \hat{W_j} + b_i + \hat{b}_j$
   2. $W, \hat{W}, b_i, \hat{b}_j$ will be the parameters
    

In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, Dataloader

In [ ]:
class Glove(nn.Module):
    # write your code snippet here
    
    pass

5) Write a function which trains the model on the frequency matrix. Ignore the 0 entries in the matrix. (15 points)

In [ ]:
# Since many entries in the matrix would be 0, it makes sense to explicitly keep track of the positive entries and iterate
# over them rather than writing a nested for loop...
# You can wrap this entries in a torch Dataset class
####################### optional ####################
class GOT_data(Dataset):
    def __init__(self):
        pass
    
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

#####################################################

# Adopt your code to incorporate mini-batch training
def train(model, data, epochs=5, learning_rate=0.001):
    # write your code snippet here
    
    pass

6) Write a function to generate embedding of a given word. Note that the embeddings of a word ($i$) would be $W_i + \hat{W}_i$ (5 points) 

# Intrinsic evaluation of embeddings

(Slide 47, lecture_4)
Word similarity task is often used as an intrinsic evaluation criteria. In the dataset file you will find a list of word pairs with their similarity scores as judged by humans. The task would be to judge how well are the word vectors aligned to human judgement. We will use word2vec embedding vectors trained on the google news corpus. (Ignore the pairs where at least one the words is absent in the corpus)

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

7) Write a function which takes as input two words and computes the cosine similarity between them. (3 points)

In [ ]:
def similarity(word1, word2):
    pass

8) Compute the similarity between all the word pairs in the list and sort them based on the similarity scores. (3 points)

9) Sort the word pairs in the list based on the human judgement scores. (2 points)

10) Compute spearman rank correlation between the two ranked lists obtained in the previous two steps. (2 points)

# Word embedding based classifier

We will design a simple sentiment classifier based on the pre-trained word embeddings (google news).

Each data point is a movie review and the sentiment could be either positive (1) or negative (0)

In [1]:
import pickle

In [3]:
with open('sentiment_test_X.p', 'rb') as fs:
    test_X = pickle.load(fs)

In [5]:
len(test_X)

1821

In [6]:
with open('sentiment_test_y.p', 'rb') as fs:
    test_y = pickle.load(fs)

In [8]:
len(test_y)

1821

In [10]:
test_X[0]

['If',
 'you',
 'sometimes',
 'like',
 'to',
 'go',
 'to',
 'the',
 'movies',
 'to',
 'have',
 'fun',
 ',',
 'Wasabi',
 'is',
 'a',
 'good',
 'place',
 'to',
 'start',
 '.']

In [11]:
test_y[0]

1

In [12]:
with open('sentiment_train_X.p', 'rb') as fs:
    train_X = pickle.load(fs)
with open('sentiment_train_y.p', 'rb') as fs:
    train_y = pickle.load(fs)
with open('sentiment_val_X.p', 'rb') as fs:
    val_X = pickle.load(fs)
with open('sentiment_val_y.p', 'rb') as fs:
    val_y = pickle.load(fs)        

11) Given a review, compute its embedding by averaging over the embedding of its constituent words. Define a function which given a review as a list of words, generates its embeddings by averaging over the constituent word embeddings. (5 points)

In [ ]:
def generate_embedding(review):
    # return embedding
    pass

12) Create a feed-forward network class with pytorch. (Hyperparamter choice such as number of layers, hidden size is left to you) (10 points)

In [ ]:
class Classifier(nn.Module):
    pass

13) Create a Dataset class for efficiently enumerating over the dataset. (5 points)

In [ ]:
class sent_data(Dataset):
    def __init__(self):
        pass
    
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

14) Write a train function to train model. At the end of each epoch compute the validation accuracy and save the model with the best validation accuracy. (15 points)

In [ ]:
# Adopt your code to incorporate mini-batch training
# Use cross-entropy as your loss function
def train(model, train_data, val_data, epochs=5, learning_rate=0.001):
    # write your code snippet here
    
    pass

15) Evaluate the trained model on the test set and report the test accuracy. (5 points)

In [13]:
def evaluate(model, test_data):
    pass